## **silver-companies-stock-news**
#### <p>This notebook extracts and ingests raw stock market news data related to publicly listed companies from Yahoo Finance. It captures headlines, timestamps, and brief summaries to build a foundational news dataset for sentiment or trend analysis.</p>

#### **Importing necessary libraries**

In [19]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, d8cd6e39-52d8-48cd-8056-b8e06bb88782, 23, Finished, Available, Finished)

#### **Variables**

In [20]:
file_name = "comp_news_data"
layer_name = "bronze"

StatementMeta(, d8cd6e39-52d8-48cd-8056-b8e06bb88782, 24, Finished, Available, Finished)

#### **Step 2: Read the Stock Companies Stock News Data**

In [21]:
com_news_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

StatementMeta(, d8cd6e39-52d8-48cd-8056-b8e06bb88782, 25, Finished, Available, Finished)

#### **Step 3: Select the necessary columns, Change the Date Column DataType, and Remove the Null Records from the News Data**

In [22]:
output_df = com_news_df\
                    .withColumn('Date', col('DisplayTime').cast(DateType()))\
                    .select(
                        'title',
                        'Date',
                        col('displayName').alias('news_agency'),
                        col('contentType').alias('content_type'),
                        col('url').alias('news_url'),
                        col('summary').alias('news_summary'),
                        'company'
                    )\
                    .filter(col('Date').isNotNull())

StatementMeta(, d8cd6e39-52d8-48cd-8056-b8e06bb88782, 26, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [23]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        latest_date = delta_table.toDF().agg({"Date": "max"}).collect()[0][0]
        new_data_to_append = output_df.filter(col("Date") > latest_date)
        
        delta_table.alias("target").merge(
            new_data_to_append.alias("source"),
            (col("target.Company") == col("source.Company")) &
            (col("target.Date") == col("source.Date"))
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

    else:
        output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, d8cd6e39-52d8-48cd-8056-b8e06bb88782, 27, Finished, Available, Finished)